In [52]:
import os
import pickle
import tarfile
import datetime
import numpy as np
import urllib.request
import sklearn.metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
import keras.utils
import tensorflow.keras.utils
import shutil

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D, Activation
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from keras.utils import plot_model
from keras import layers, Model
import keras

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

I0000 00:00:1729299083.309588  233692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729299083.332220  233692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729299083.332280  233692 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
%load_ext tensorboard

In [69]:
BATCH_SIZE = 128
HISTORY_DIR = './history'
os.makedirs(HISTORY_DIR, exist_ok=True)

In [12]:
def download_data():
    if not os.path.exists('tiny-imagenet-200'):
        urllib.request.urlretrieve('http://cs231n.stanford.edu/tiny-imagenet-200.zip', 'tiny-imagenet-200.zip')
        file = zipfile.ZipFile('tiny-imagenet-200.zip', 'r')
        file.extractall()

In [14]:
download_data()

In [57]:
# Формирование директории валидационных данных

def create_validation_dir():
    print('Start creating validation dir')

    ANNOTATIONS_PATH = './tiny-imagenet-200/val/val_annotations.txt'

    file_name_label_pairs = []
    
    with open(ANNOTATIONS_PATH) as annotation_file:
        for line in annotation_file:
            annotaion_item_data = line.split('\t')
            file_name, label = annotaion_item_data[0], annotaion_item_data[1]

            file_name_label_pairs.append([file_name, label])

    file_name_label_pairs = np.array(file_name_label_pairs)
    
    labels = os.listdir('./tiny-imagenet-200/train')

    print(f'Create {len(labels)} label dir')
    os.mkdir(os.getcwd() + '/tiny-imagenet-200/validation')
        
    for label in labels:
        os.mkdir(os.getcwd() + '/tiny-imagenet-200/validation/' + label)

    for i in range(0, len(file_name_label_pairs)):
        if i % 1000 == 0:
            print(f'Process {i} files now {file_name_label_pairs[i]}')
        
        shutil.copy2(
            './tiny-imagenet-200/val/images/' + file_name_label_pairs[i][0],
            './tiny-imagenet-200/validation/' + file_name_label_pairs[i][1] + '/' + file_name_label_pairs[i][0] 
        )
    
    print('End creating validation dir')

In [58]:
create_validation_dir()

Start creating validation dir
Create 200 label dir
Process 0 files now ['val_0.JPEG' 'n03444034']
Process 1000 files now ['val_1000.JPEG' 'n04023962']
Process 2000 files now ['val_2000.JPEG' 'n03992509']
Process 3000 files now ['val_3000.JPEG' 'n03977966']
Process 4000 files now ['val_4000.JPEG' 'n03100240']
Process 5000 files now ['val_5000.JPEG' 'n02074367']
Process 6000 files now ['val_6000.JPEG' 'n03891332']
Process 7000 files now ['val_7000.JPEG' 'n04532670']
Process 8000 files now ['val_8000.JPEG' 'n02058221']
Process 9000 files now ['val_9000.JPEG' 'n02002724']
End creating validation dir


In [64]:
random_seed = 42
batch_size = 64
EPOCHS = 60
image_size = (64, 64)

In [61]:
train_dataset = keras.utils.image_dataset_from_directory(
    './tiny-imagenet-200/train/',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=random_seed,
    verbose=True,
)

Found 100000 files belonging to 200 classes.


I0000 00:00:1729308100.663841  240362 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729308101.061287  240362 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729308101.061346  240362 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729308101.064386  240362 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729308101.064438  240362 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [63]:
validation_dataset = keras.utils.image_dataset_from_directory(
    './tiny-imagenet-200/validation/',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=random_seed,
    verbose=True,
)

Found 10000 files belonging to 200 classes.


In [70]:
INPUT_SHAPE = (64, 64, 3)

input = Input(shape=INPUT_SHAPE)

x = keras.layers.RandomFlip("horizontal")(input)
x = keras.layers.RandomRotation(0.1)(x)

x = Conv2D(32, 5, activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = Conv2D(32, 5, activation='relu', padding="same")(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, 3, activation='relu', padding="same")(x)
x = Conv2D(64, 3, activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, 3, activation='relu', padding="same")(x)
x = Conv2D(128, 3, activation='relu', padding="same")(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
output = Dense(200, activation="softmax")(x)

model = Model(input, output)

In [71]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_1 (RandomFlip)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (None, 64, 64, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 200)            │        51,400 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,343,464 (8.94 MB)

 Trainable params: 2,343,272 (8.94 MB)

 Non-trainable params: 192 (768.00 B)

In [72]:
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="Adam")

In [73]:
logdir = os.path.join(HISTORY_DIR, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [74]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(logdir, 'model.keras'),
    save_best_only=True
)

In [75]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(logdir, 'logs.keras'),    
)

In [76]:
%tensorboard --logdir $logdir

UsageError: Line magic function `%tensorboard` not found.


In [77]:
keras.utils.set_random_seed(random_seed)


history =  model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS, callbacks=[model_checkpoint_callback, tensorboard_callback])

Epoch 1/60


2024-10-19 13:24:56.651283: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1729308296.975969  242494 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308297.028940  242494 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308297.058764  242494 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308297.103437  242494 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308297.133229  242494 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308297.170746  242494 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308297.197028  242494 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308297.230357  242494 gpu_t

1562/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - accuracy: 0.0321 - loss: 4.9907

W0000 00:00:1729308880.342135  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.349529  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.356998  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.364392  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.372102  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.380102  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.388060  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.397940  242496 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308880.407834  242496 gp

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step - accuracy: 0.0321 - loss: 4.9905

W0000 00:00:1729308884.494365  242491 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.601090  242492 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.609121  242492 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.615755  242492 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.622477  242492 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.629104  242492 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.636274  242492 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.642967  242492 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729308897.650933  242492 gp

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 605s 380ms/step - accuracy: 0.0322 - loss: 4.9904 - val_accuracy: 0.0809 - val_loss: 4.4982
Epoch 2/60
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 587s 375ms/step - accuracy: 0.1120 - loss: 4.2382 - val_accuracy: 0.1186 - val_loss: 4.3248
Epoch 3/60
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 596s 381ms/step - accuracy: 0.1623 - loss: 3.8788 - val_accuracy: 0.1639 - val_loss: 3.9608
Epoch 4/60
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 593s 380ms/step - accuracy: 0.1944 - loss: 3.6503 - val_accuracy: 0.1946 - val_loss: 3.6672
Epoch 5/60
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 598s 382ms/step - accuracy: 0.2213 - loss: 3.4985 - val_accuracy: 0.2049 - val_loss: 3.6299
Epoch 6/60
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 655s 419ms/step - accuracy: 0.2428 - loss: 3.3692 - val_accuracy: 0.2340 - val_loss: 3.4287
Epoch 7/60
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 635s 406ms/step - accuracy: 0.2641 - loss: 3.2542 - val_accuracy: 0.1961 - val_loss: 3.8655
Epoch 8/60
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 641s 410ms/step - accuracy: 0.2

In [78]:
# Best model
best_model = keras.saving.load_model('history/20241019-132438/model.keras')

In [83]:
benchmark = best_model.evaluate(validation_dataset)
print(benchmark)

157/157 ━━━━━━━━━━━━━━━━━━━━ 14s 86ms/step - accuracy: 0.2925 - loss: 3.2267
[3.236783027648926, 0.29409998655319214]


In [103]:
prdiction = best_model.predict(validation_dataset)

157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 86ms/step


In [101]:
# as_numpy_iterator

y_true = np.array([item[1] for item in validation_dataset.unbatch().as_numpy_iterator()])
y_pred = np.array([np.argmax(item) for item in prdiction])

In [102]:
_, ax = plt.subplots(figsize=(75, 75))
sklearn.metrics.ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax, colorbar=False)

plt.tight_layout()
plt.savefig('valid.png')